In [7]:
# insert system_id
conn = psycopg2.connect(dbname='youdoDB', user='postgres', 
                        password='1234', host='localhost', 
                        port='5432')
cur = conn.cursor()

acc_query = 'INSERT INTO account(state, is_blocked, created_on)'\
' VALUES (0, false, now())'
cur.execute(acc_query)

conn.commit()
cur.close()
conn.close()

In [4]:
# filling experience, account and user tables

import random
import psycopg2
from datetime import datetime as DT
from datetime import timedelta

def get_random_date(start, end):
    delta = end - start
    return start + timedelta(random.randint(0, delta.days))

USER_CNT = 0

def get_user_query(exp_id, acc_id):
    NAME_M = ['Антон', 'Андрей', 'Иван', 'Григорий', 'Александр', 
              'Никита', 'Руслан', 'Кирилл', 'Ибрагим', 'Анатолий', 
              'Михаил', 'Денис']
    NAME_F = ['Анастасия', 'Татьяна', 'Надежда', 'Марина', 
              'Екатерина', 'Ирина', 'Анна', 'Мария', 'Дарья']
    SURNAME = ['Иванов','Петров','Кочетков','Гончаров','Печкин',
               'Абрамов','Львов','Задорнов','Измайлов','Баранов',
               'Комаров','Лыткарин','Морев','Колчанов','Смирнов',
               'Харитонов','Ворошилов']
    LOGIN = ['cat', 'fox', 'owl', 'bear', 'parrot', 'pen', 
             'puma', 'tiger', 'lion', 'ostrich', 'sheep', 'dog', 
             'cow', 'shrek', 'shadow']
    EMAIL = ['@gmail.com', '@yandex.ru', '@mail.ru', '@mephi.ru']
    query = 'INSERT INTO "user" (full_name, birth_date, gender, '\
    'email, phone_number, documents_checked, profile_photo, '\
    'created_on, last_login, experience_id, account_id) VALUES'

    m_flag = random.randint(0, 1)
    
    if (m_flag == 0):
        name = random.choice(NAME_M)+' '+random.choice(SURNAME)
    else:
        name = random.choice(NAME_F)+' '+random.choice(SURNAME)+'а'
    
    start_dt = DT.strptime('01.01.1930', '%d.%m.%Y')
    end_dt = DT.strptime('01.01.2005', '%d.%m.%Y')

    birth_date = get_random_date(start_dt, end_dt)
    email = random.choice(LOGIN)+str(USER_CNT)+random.choice(EMAIL)
    phone_number = '89'+random.choice(['25','78','80','09','04'])+str(random.randint(1000000, 9999999))
    
    if (random.randint(0,1) == 0):
        documents_checked = 'false'
    else:
        documents_checked = 'true'
    
    if (m_flag == 0):
        query += f'(\'{name}\', \'{birth_date}\', \'0\', '\
        '\'{email}\', \'{phone_number}\', \'{documents_checked}\','\
        ' null, now(), now(), {exp_id}, {acc_id})'
    else:
        query += f'(\'{name}\', \'{birth_date}\', \'1\', '\
        '\'{email}\', \'{phone_number}\', \'{documents_checked}\', '\
        'null, now(), now(), {exp_id}, {acc_id})'
    query += ' RETURNING user_id'
    
    return query

conn = psycopg2.connect(dbname='youdoDB', user='postgres', 
                        password='1234', host='localhost', 
                        port='5432')
cur = conn.cursor()
 
for i in range(100):
 
    exp_query = 'INSERT INTO experience(experience_age, about, '\
    'business_status, video_link, created_on) VALUES (default, '\
    'default, default, default, now()) RETURNING experience_id'
    acc_query = 'INSERT INTO account(state, is_blocked, created_on)'\
    ' VALUES (0, false, now()) RETURNING account_id'
  
    cur.execute(exp_query)
    exp_id = cur.fetchall()[0][0]
    cur.execute(acc_query)
    acc_id = cur.fetchall()[0][0]
        
    user_query = get_user_query(exp_id, acc_id)
    res = cur.execute(user_query)
    USER_CNT += 1
    
conn.commit()
cur.close()
conn.close()

In [5]:
# fill albums

import random
import psycopg2

experience_ids_query = "SELECT experience_id FROM experience"
conn = psycopg2.connect(dbname='youdoDB', user='postgres', 
                        password='1234', host='localhost', 
                        port='5432')
cur = conn.cursor()
cur.execute(experience_ids_query)
res = cur.fetchall()
ids = []
for i in res:
    ids.append(i[0])

def get_album_query(ids):
    NAME_1 = ["Good", "Hard", "Excellent", "Tiny", "Cute", "Grand", 
              "Ideal", "Default", "Important", "Quick"]
    NAME_2 = ["work", "task", "experience", "job", "diagnosis",
              "deal"]
    name = random.choice(NAME_1) + " " + random.choice(NAME_2)
    des = "This was really " + random.choice(NAME_1).lower() + 
        " " + random.choice(NAME_2) + " that took " + 
        str(random.randint(2, 10)) + " " + 
        random.choice(["hours", "days", "months"]) + "."
    id = random.choice(ids)
    query = "INSERT INTO album(\"name\", description, created_on,"\
            " experience_id) VALUES "
    query += f'(\'{name}\', \'{des}\', now(), \'{id}\')'
    return query

for i in range(100):
    query = get_album_query(ids)
    cur.execute(query)
    
conn.commit()
cur.close()
conn.close()


In [26]:
# fill account

import random
import psycopg2

conn = psycopg2.connect(dbname='youdoDB', user='postgres', 
                        password='1234', host='localhost', 
                        port='5432')
cur = conn.cursor()

account_ids_query = "SELECT account_id FROM account"
get_system_account_query = "SELECT account_id FROM account "\
"WHERE NOT EXISTS (SELECT account_id FROM \"user\" WHERE "\
"\"user\".account_id=account.account_id)"
cur.execute(get_system_account_query)
system_id = cur.fetchall()[0][0]

cur.execute(account_ids_query)
res = cur.fetchall()
ids = []
for i in res:
    if(i[0] == system_id):
        continue
    ids.append(i[0])

# "пополнение счета"
for i in range(100):
    id = random.choice(ids)
    money = random.randint(40,100)*10
    cur.execute(f"SELECT \"state\"::numeric::int FROM account "\
                " WHERE account_id={id}")
    before = cur.fetchall()[0][0]
    account_query = f"UPDATE account SET "\
    "\"state\"={int(before) + money} WHERE account_id={id}"
    transaction_query = f"INSERT INTO \"transaction\""\
    "(\"time\", \"size\", created_on, debit_account_id,"\
    " replenishment_account_id, transaction_status) VALUES "\
    "(default, {money}, default, {system_id}, {id}, 'done')" 
    cur.execute(account_query)
    cur.execute(transaction_query)

conn.commit()
cur.close()
conn.close()

In [17]:
# pay to system

import random
import psycopg2

conn = psycopg2.connect(dbname='youdoDB', user='postgres', password='1234', 
                        host='localhost', port='5432')
cur = conn.cursor()

get_system_account_query = "SELECT account_id FROM account WHERE NOT EXISTS "\
"(SELECT account_id FROM \"user\" WHERE \"user\".account_id=account.account_id)"
cur.execute(get_system_account_query)
system_id = cur.fetchall()[0][0]

for i in range(100):    
    account_ids_query = f"SELECT account_id FROM account WHERE "\
    "\"state\"::numeric::int >= 10"
    cur.execute(account_ids_query)
    res = cur.fetchall()
    ids = []
    for i in res:
        if(i[0] == system_id):
            continue
        ids.append(i[0])    
    id = random.choice(ids)
    cur.execute(f"SELECT \"state\"::numeric::int FROM account WHERE "\
                " account_id={id}")
    before = cur.fetchall()[0][0]
    
    money = random.randint(10, before)
    
    cur.execute(f"SELECT \"state\"::numeric::int FROM account WHERE "\
                " account_id={system_id}")
    system_before = cur.fetchall()[0][0]
    account_query = f"UPDATE account SET \"state\"={int(before) - money}"\
                    f" WHERE account_id={id}"
    system_query = f"UPDATE account SET \"state\"={int(system_before) + money}"\
                    f" WHERE account_id={system_id}"
    transaction_query = f"INSERT INTO \"transaction\"(\"time\", \"size\", "\
    " created_on, debit_account_id, replenishment_account_id, transaction_status)"\
    f" VALUES (default, {money}, default, {id}, {system_id}, 'done')" 
    cur.execute(account_query)
    cur.execute(system_query)
    cur.execute(transaction_query)

conn.commit()
cur.close()
conn.close()

In [25]:
# payment between users

import random
import psycopg2

conn = psycopg2.connect(dbname='youdoDB', user='postgres', password='1234', 
                        host='localhost', port='5432')
cur = conn.cursor()

# get system_id
get_system_account_query = "SELECT account_id FROM account WHERE NOT EXISTS "\
"(SELECT account_id FROM \"user\" WHERE \"user\".account_id=account.account_id)"
cur.execute(get_system_account_query)
system_id = cur.fetchall()[0][0]
# select replenishment ids
account_ids_query2 = "SELECT account_id FROM account"
cur.execute(account_ids_query2)
res = cur.fetchall()
ids2 = []
for i in res:
    if(i[0] == system_id):
        continue
    ids2.append(i[0])
# start filling
for i in range(30):
    id2 = random.choice(ids2)
    # select debit id
    account_ids_query1 = f"SELECT account_id FROM account WHERE "\
    f"account.\"state\"::numeric::int > 500 AND account.account_id != {id2}"
    cur.execute(account_ids_query1)
    res = cur.fetchall()
    ids1 = []
    for i in res:
        if(i[0] == system_id):
            continue
        ids1.append(i[0])
    id1 = random.choice(ids1)
    # select accounts' states
    cur.execute(f"SELECT \"state\"::numeric::int FROM account WHERE "\
                f"account_id={id1}")
    before1 = cur.fetchall()[0][0]
    money = random.randint(5, int(before1 / 100))*100
    cur.execute(f"SELECT \"state\"::numeric::int FROM account "\
                f"WHERE account_id={id2}")
    before2 = cur.fetchall()[0][0]
    account1_query = f"UPDATE account SET \"state\"={int(before1) - money} "\
                    f" WHERE account_id={id1}"
    account2_query = f"UPDATE account SET \"state\"={int(before2) + money} "\
                    f"WHERE account_id={id2}"
    if(random.randint(0, 100) < 30):
        status = "freezing money"
    else:
        status = "done" 
    transaction_query = f"INSERT INTO \"transaction\"(\"time\", \"size\", "\
    " created_on, debit_account_id, replenishment_account_id, "\
    f" transaction_status) VALUES (default, {money}, default, {id1}, {id2}, "\
    f"'{status}')" 
    cur.execute(account1_query)
    cur.execute(account2_query)
    cur.execute(transaction_query)

conn.commit()
cur.close()
conn.close()

In [28]:
# failed payment between users

import random
import psycopg2

conn = psycopg2.connect(dbname='youdoDB', user='postgres', password='1234', 
                        host='localhost', port='5432')
cur = conn.cursor()

# get system_id
get_system_account_query = "SELECT account_id FROM account WHERE NOT EXISTS "\
"(SELECT account_id FROM \"user\" WHERE \"user\".account_id=account.account_id)"
cur.execute(get_system_account_query)
system_id = cur.fetchall()[0][0]
# get ids
experience_ids_query = "SELECT account_id FROM account"
cur.execute(experience_ids_query)
res = cur.fetchall()
ids = []
for i in res:
    if(i[0] == system_id):
        continue
    ids.append(i[0])

for i in range(10):
    money = random.randint(5, 100)*100
    id2 = random.choice(ids2)
    id1 = random.choice(ids1)
    transaction_query = f"INSERT INTO \"transaction\"(\"time\", \"size\", "\
    "created_on, debit_account_id, replenishment_account_id, transaction_status)"\
    f" VALUES (default, {money}, default, {id1}, {id2}, 'failed')" 
    cur.execute(transaction_query)

conn.commit()
cur.close()
conn.close()

In [ ]:
delete from "user";
delete from album;
delete from experience cascade;
delete from "transaction";
delete from account;
delete from "user";

In [1]:
# filling categories
import random
import psycopg2

conn = psycopg2.connect(dbname='youdoDB', user='postgres', password='1234', 
                        host='localhost', port='5432')
cur = conn.cursor()

cat_names = ["courier service", "repair and construction", "trucking",
             "cleaning and housekeeping assistance", "virtual assistant", 
             "computer help", "events and promotions", "design", 
             "software development", "photo, video and audio", 
             "installation and repair of equipment", "beauty and health",
             "digital equipment repair", "legal and accounting assistance",
             "contracts, documents, courts, taxes", "transport repair"]
cat_desc = ["delivery", "build or repair", "cargo transportation", 
            "home care", "help at a distance or not", 
            "installation, removal, fix, explanation", 
            "organization and preparation", "creative solutions", 
            "the work of programmers", "media content specialists", 
            "fix or install something", "your body and appearance", 
            "repair of PC, laptops, phones", 
            "contracts, documents, courts, taxes", 
            "development, education, sport, driving", "mechanic help"]

for i in range(0, len(cat_names)):
    query = f"INSERT INTO category(\"name\", description, created_on)"\
            f" VALUES ('{cat_names[i]}', '{cat_desc[i]}', default);"
    cur.execute(query)
    
conn.commit()
cur.close()
conn.close()

In [15]:
# filling estimated cost for all categories

conn = psycopg2.connect(dbname='youdoDB', user='postgres', password='1234',
                        host='localhost', port='5432')
cur = conn.cursor()

arr = [17, 18, 19, 20, 21,22,23,24,25,26,27,28,29,30,31,32]


# courier service
query = f"INSERT INTO estimated_cost(interval_start, interval_end, "\
"response_cost, category_id, created_on) VALUES "\
f"(500, 600, 29, {arr[0]}, default), "\
f"(null, 500, 11, {arr[0]}, default), "\
f"(800, 1200, 60, {arr[0]}, default), "\
f"(1200, 1500, 99, {arr[0]}, default), "\
f"(1500, 2500, 141, {arr[0]}, default), "\
f"(2500, null, 199, {arr[0]}, default), "\
f"(600, 800, 42, {arr[0]}, default); "
cur.execute(query)

# repair and construction
query = f"INSERT INTO estimated_cost(interval_start, interval_end, "\
f" response_cost, category_id, created_on) VALUES "\
f"(null, 3000, 62, {arr[1]}, default), "\
f"(3000, 5000, 80, {arr[1]}, default), "\
f"(5000, 10000, 100, {arr[1]}, default), "\
f"(10000, 36000, 137, {arr[1]}, default), "\
f"(36000, null, 200, {arr[1]}, default); "
cur.execute(query)

# trucking
query = f"INSERT INTO estimated_cost(interval_start, interval_end, "\
f"response_cost, category_id, created_on) VALUES "\
f"(null, 2000, 30, {arr[2]}, default), "\
f"(2000, 5000, 50, {arr[2]}, default), "\
f"(5000, 8000, 80, {arr[2]}, default), "\
f"(8000, 15000, 100, {arr[2]}, default), "\
f"(15000, 24000, 131, {arr[2]}, default), "\
f"(24000, null, 199, {arr[2]}, default);"
cur.execute(query)

# cleaning and housekeeping assistance
query = f"INSERT INTO estimated_cost(interval_start, interval_end, response_cost, category_id, created_on) VALUES "\
f"(null, 1500, 38, {arr[3]}, default), "\
f"(1500, 3000, 53, {arr[3]}, default), "\
f"(3000, 5000, 76, {arr[3]}, default), "\
f"(5000, 8000, 91, {arr[3]}, default), "\
f"(8000, null, 109, {arr[3]}, default);"
cur.execute(query)

# virtual assistant
query = f"INSERT INTO estimated_cost(interval_start, interval_end, response_cost, category_id, created_on) VALUES "\
f"(null, 1000, 20, {arr[4]}, default), "\
f"(1000, 3000, 50, {arr[4]}, default), "\
f"(3000, 8000, 100, {arr[4]}, default), "\
f"(8000, 10000, 155, {arr[4]}, default), "\
f"(10000, null, 250, {arr[4]}, default);"
cur.execute(query)

# computer help
query = f"INSERT INTO estimated_cost(interval_start, interval_end, response_cost, category_id, created_on) VALUES "\
f"(null, 500, 30, {arr[5]}, default), "\
f"(500, 1500, 50, {arr[5]}, default), "\
f"(1500, 2500, 66, {arr[5]}, default), "\
f"(2500, 3500, 81, {arr[5]}, default), "\
f"(3500, null, 101, {arr[5]}, default);"
cur.execute(query)

# events and promotions
query = f"INSERT INTO estimated_cost(interval_start, interval_end, response_cost, category_id, created_on) VALUES "\
f"(null, 500, 22, {arr[6]}, default), "\
f"(500, 3500, 50, {arr[6]}, default), "\
f"(3500, 5000, 74, {arr[6]}, default), "\
f"(5000, 10000, 98, {arr[6]}, default), "\
f"(10000, null, 124, {arr[6]}, default);"
cur.execute(query)

# design
query = f"INSERT INTO estimated_cost(interval_start, interval_end, response_cost, category_id, created_on) VALUES "\
f"(null, 500, 26, {arr[7]}, default), "\
f"(500, 1500, 36, {arr[7]}, default), "\
f"(1500, 5000, 54, {arr[7]}, default), "\
f"(5000, 10000, 98, {arr[7]}, default), "\
f"(10000, 50000, 150, {arr[7]}, default), "\
f"(50000, null, 299, {arr[7]}, default);"
cur.execute(query)

# software development
query = f"INSERT INTO estimated_cost(interval_start, interval_end, response_cost, category_id, created_on) VALUES "\
f"(null, 1500, 57, {arr[8]}, default), "\
f"(1500, 5000, 80, {arr[8]}, default), "\
f"(5000, 10000, 90, {arr[8]}, default), "\
f"(10000, 25000, 122, {arr[8]}, default), "\
f"(25000, 50000, 150, {arr[8]}, default), "\
f"(50000, 100000, 180, {arr[8]}, default), "\
f"(100000, 150000, 201, {arr[8]}, default), "\
f"(150000, null, 299, {arr[8]}, default);"
cur.execute(query)

# photo, video and audio
query = f"INSERT INTO estimated_cost(interval_start, interval_end, response_cost, category_id, created_on) VALUES "\
f"(null, 500, 57, {arr[9]}, default), "\
f"(500, 1500, 69, {arr[9]}, default), "\
f"(1500, 3000, 90, {arr[9]}, default), "\
f"(3000, 6000, 126, {arr[9]}, default), "\
f"(6000, 15000, 185, {arr[9]}, default), "\
f"(15000, 25000, 256, {arr[9]}, default), "\
f"(25000, null, 298, {arr[9]}, default);"
cur.execute(query)

# installation and repair of equipment
query = f"INSERT INTO estimated_cost(interval_start, interval_end, response_cost, category_id, created_on) VALUES "\
f"(null, 1000, 53, {arr[10]}, default), "\
f"(1000, 2000, 70, {arr[10]}, default), "\
f"(2000, 5000, 90, {arr[10]}, default), "\
f"(5000, 10000, 134, {arr[10]}, default), "\
f"(10000, 15000, 187, {arr[10]}, default), "\
f"(15000, 30000, 256, {arr[10]}, default), "\
f"(30000, null, 289, {arr[10]}, default);"
cur.execute(query)

# beauty and health
query = f"INSERT INTO estimated_cost(interval_start, interval_end, response_cost, category_id, created_on) VALUES "\
f"(null, 500, 32, {arr[11]}, default), "\
f"(500, 1500, 50, {arr[11]}, default), "\
f"(1500, 3000, 70, {arr[11]}, default), "\
f"(3000, 5000, 121, {arr[11]}, default), "\
f"(5000, 15000, 240, {arr[11]}, default), "\
f"(15000, null, 300, {arr[11]}, default);"
cur.execute(query)

# digital equipment repair
query = f"INSERT INTO estimated_cost(interval_start, interval_end, response_cost, category_id, created_on) VALUES "\
f"(null, 1500, 34, {arr[12]}, default), "\
f"(1500, 3000, 58, {arr[12]}, default), "\
f"(3000, 10000, 70, {arr[12]}, default), "\
f"(10000, null, 123, {arr[12]}, default);"
cur.execute(query)

# legal and accounting assistance
query = f"INSERT INTO estimated_cost(interval_start, interval_end, response_cost, category_id, created_on) VALUES "\
f"(null, 500, 34, {arr[13]}, default), "\
f"(500, 1500, 58, {arr[13]}, default), "\
f"(1500, 5000, 100, {arr[13]}, default), "\
f"(5000, 7000, 165, {arr[13]}, default), "\
f"(7000, 15000, 241, {arr[13]}, default), "\
f"(15000, 40000, 357, {arr[13]}, default), "\
f"(40000, null, 456, {arr[13]}, default);"
cur.execute(query)

# contracts, documents, courts, taxes
query = f"INSERT INTO estimated_cost(interval_start, interval_end, response_cost, category_id, created_on) VALUES "\
f"(null, 1000, 50, {arr[14]}, default), "\
f"(1000, 3000, 123, {arr[14]}, default), "\
f"(3000, 10000, 254, {arr[14]}, default), "\
f"(10000, null, 368, {arr[14]}, default);"
cur.execute(query)

# transport repair
query = f"INSERT INTO estimated_cost(interval_start, interval_end, response_cost, category_id, created_on) VALUES "\
f"(null, 1500, 50, {arr[15]}, default), "\
f"(1500, 5000, 83, {arr[15]}, default), "\
f"(5000, 10000, 124, {arr[15]}, default), "\
f"(10000, 50000, 250, {arr[15]}, default), "\
f"(50000, null, 400, {arr[15]}, default);"
cur.execute(query)

conn.commit()
cur.close()
conn.close()

In [109]:
# fill review

import random
import psycopg2

conn = psycopg2.connect(dbname='youdoDB', user='postgres', password='1234',
                        host='localhost', port='5432')
cur = conn.cursor()

# get ids
user_ids_query = "SELECT user_id FROM \"user\""
cur.execute(user_ids_query)
res = cur.fetchall()
ids = []
for i in res:
    ids.append(i[0])
    
for i in range(100):
    id1 = random.choice(ids)
    id2 = random.choice(ids)
    while (id1 == id2):
        id2 = random.choice(ids)
    NAME_Good1 = ["Good", "Excellent", "Cute", "Grand", "Ideal", "Important", 
                  "Quick"]
    NAME_Bad1 = ["Awfull", "Hard", "Useless", "So expensive", "Really bad", 
                 "Bad", "The worst", "Failed", "Important", "Long"]
    NAME_2 = ["work", "task", "experience", "job", "diagnosis", "deal"]
    START_Bad = ["So", "Unfortunately", "I am angry because", "To be honest,"]
    START_Good = ["So", "Fortunately", "I am happy because", "Great that"]
    END = ["pay for this again", "recommend this performer", 
           "use this service again", "choose this performer again"]
    GRADE = ["1", "2", "3", "4", "5"]
    flag1 = random.randint(0, 100)
    if (flag1 > 20):
        grade1 = random.choice(GRADE[2:5])
        grade2 = random.choice(GRADE[3:5])
        grade3 = random.choice(GRADE[2:5])
        punc = random.choice([".", ")", ")!", "))))" "!"])
        description = f"{random.choice(START_Good)} it was "\
        f"{random.choice(NAME_Good1).lower()} {random.choice(NAME_2)} "\
        f"and I will {random.choice(END)}{punc}"
    else:
        grade1 = random.choice(GRADE[0:2])
        grade2 = random.choice(GRADE[0:4])
        grade3 = random.choice(GRADE[0:4])
        punc = random.choice(["...", ".", "(", "(((" "!", "!!!"])
        description = f"{random.choice(START_Bad)} it was "\
        f"{random.choice(NAME_Bad1).lower()} {random.choice(NAME_2)} "\
        f"and I will not {random.choice(END)}{punc}"
    review_query = f"INSERT INTO review"\
    f"(description, quality_grade, politeness_grade, cost_grade, created_on,"\
    " owner_id, author_id) VALUES "\
    f"(\'{description}\', '{grade1}', '{grade2}', '{grade3}', default, "\
    f" {id1}, {id2})" 
    cur.execute(review_query)

conn.commit()
cur.close()
conn.close()

In [77]:
# fill task!

import random
import psycopg2
import pandas

conn = psycopg2.connect(dbname='youdoDB', user='postgres', password='1234', host='localhost', port='5432')
cur = conn.cursor()

# get user ids
user_ids_query = "SELECT user_id FROM \"user\""
cur.execute(user_ids_query)
res = cur.fetchall()
ids = []
for i in res:
    ids.append(i[0])
    
# get category_id and estimated_cost_id
query = "SELECT estimated_cost_id, category_id FROM estimated_cost"
cur.execute(query)
res = cur.fetchall()
pair_ids = []
for i in res:
    pair_ids.append(i)

for i in range(100):
    # choosing task status, author and performer ids
    if(random.randint(0, 100) > 60):
        id2 = "null"
        task_status = "posted"
    else:
        id2 = random.choice(ids)
        if(random.randint(1, 100) > 20):
            task_status = "processing"
        else:
            task_status = "done"
    id1 = random.choice(user_ids)
    while (id2 == id1):
        id1 = random.choice(user_ids)
    # choosing category and cost
    estimated_cost_id, category_id = random.choice(pair_ids)
    # choose start and end time
    start_time = str(random.choice(pandas.date_range('2022-01-01', '2022-12-31')))[:-9]
    end_time = str(random.choice(pandas.date_range(start_time, 
                                                   '2022-12-31')))[:-9]
    #generate name
    NAME_1 = ["Tricky", "Unusual", "Good", "Hard", "Grand", "Careful", 
              "Quick", "Important", "Easy", "Small", "Big"]
    NAME_2 = ["work", "task", "job"]
    name = random.choice([f"{random.choice(NAME_1)} {random.choice(NAME_2)}", 
                          f"Help with {random.choice(NAME_1).lower()}"\
                          f" {random.choice(NAME_2)}"])
    description = f"I need {random.choice(['help to', 'a professional to','a person to', 'to'])} "\
                f"do some {random.choice(NAME_1).lower()} {random.choice(NAME_2)} "\
                f"and also I {random.choice(['need materials', 'have busy schedule','need to get some advices','have an angry dog','live far from metro', 'do not have much time'])}"
    place = random.choice(["at home", "distant", "Kashirskaya", 
                           "Technopark", "Park Gorkiy", "Mayakovskaya", 
                           "Taganskaya", "Kolomenskaya", "Paveletskaya", 
                           "Marksistskaya", "Aviamotornaya", "Vihino", 
                           "Tsaritsino", "Lublino", "Yugo-Zapadnaya", "VDNH", 
                           "Biblioteka", "Tretyakovskaya", "Ploschad Ilyicha"])
    task_query = f"INSERT INTO task(\"name\", description, begin_time, "\
    f"end_time, place, \"task_status\", created_on, author_id, performer_id, "\
    " category_id, estimated_cost_id) VALUES "\
    f"('{name}', \'{description}\', '{start_time}', '{end_time}', '{place}', "\
    f" '{task_status}', default, {id1}, {id2}, {category_id}, "\
    f"{estimated_cost_id})" 
    cur.execute(task_query)

conn.commit()
cur.close()
conn.close()

In [53]:
# fill response

import random
import psycopg2
import pandas

conn = psycopg2.connect(dbname='youdoDB', user='postgres', password='1234', 
                        host='localhost', port='5432')
cur = conn.cursor()

# get user ids
user_ids_query = "SELECT user_id FROM \"user\""
cur.execute(user_ids_query)
res = cur.fetchall()
user_ids = []
for i in res:
    user_ids.append(i[0])
    
# get category_id and estimated_cost_id
query = "SELECT task_id, author_id, estimated_cost_id FROM task"
cur.execute(query)
res = cur.fetchall()
triple = []
for i in res:
    triple.append(i)

for i in range(100):
    task_id, author_id, estimated_cost_id = random.choice(triple)
    cur.execute(f"SELECT interval_start::numeric::int, "\
                f"interval_end::numeric::int FROM estimated_cost "\
                f" WHERE estimated_cost_id = {estimated_cost_id}")
    cost_start, cost_end = cur.fetchall()[0]
    if(str(cost_start) == "None"):
        cost_start = 0
    if(str(cost_end) == "None"):
        cost_end = 500000
    cost = int(cost_start) + random.randint(1, (int(cost_end) - 
                                                int(cost_start))/50)*50
    perf_id = author_id
    while (perf_id == author_id):
        perf_id = random.choice(user_ids)
    #generate description
    GREETING = ['Hello', 'Good morning', 'Good evening', 'Good afternoon', 
                'Have a nice day', 'Greetings']
    PART1 = ['I am ready to do this', 'I am glad to help you with your', 
             'I am professional in this sphere so I will do the']
    NAME_2 = ['work', 'task', 'job', 'project']
    ENDING = ['Hope you choose me', 'I am waiting for detailes', 
              'I will be waiting for your respond']
    description = random.choice(GREETING) + random.choice(['.', '!']) +  " " + random.choice([":)", "", "X)", "8)"]) + " " + random.choice(PART1) + " " + random.choice(NAME_2) + ". " +  random.choice(ENDING) + random.choice(['.', '...', '!'])
    response_query = f"INSERT INTO response"\
    f"(\"cost\", description, created_on, task_id, author_id) VALUES "\
    f"({cost}, \'{description}\', default, {task_id}, {perf_id})" 
    cur.execute(response_query)

conn.commit()
cur.close()
conn.close()